In [ ]:
!pip install requests

In [ ]:
import requests
import pandas as pd
import json
from time import sleep

In [ ]:
# パラメーター

get_all = False
word = '我'

In [ ]:
params = {"order":"released", "q":'lang:zhs ft:' + word}

response = requests.get("https://api.scryfall.com/cards/search", params=params)

response_json = json.loads(response.text)

In [ ]:
print(json.loads(response.text))

In [ ]:
len(response_json["data"]), response_json["data"][0]

In [ ]:
df = pd.DataFrame({
  "jname":[],
  "cname":[],
  "ename":[],
  "jflavor":[],
  "cflavor":[],
  "eflavor":[]
})

df

In [ ]:
while True:
  for i, card in enumerate(response_json["data"]):
    if(i % 10 == 0): print(i)

    cname = card["printed_name"] if "printed_name" in card else card["name"]
    
    if not "flavor_text" in card: continue
    cflavor = card["flavor_text"]

    jresponse = requests.get("https://api.scryfall.com/cards/" + card["set"] + "/" + card["collector_number"] + "/" + "ja")
    jresponse_json = json.loads(jresponse.text)
    
    if len(jresponse_json) == 0: continue

    jcard = jresponse_json
    if not "flavor_text" in jcard: continue

    jname = jcard["printed_name"] if "printed_name" in jcard else jcard["name"]

    sleep(0.05)

    eresponse = requests.get("https://api.scryfall.com/cards/" + card["set"] + "/" + card["collector_number"] + "/" + "en")
    eresponse_json = json.loads(eresponse.text)
    
    if len(eresponse_json) == 0: continue

    ecard = eresponse_json
    if not "flavor_text" in ecard: continue

    ename = ecard["printed_name"] if "printed_name" in ecard else ecard["name"]

    sleep(0.05)

    df = pd.concat([df, pd.DataFrame({"jname": [jname], "cname":[cname], "ename":[ename] , 
                                      "jflavor": [jcard["flavor_text"].replace("\n", "\\n")], 
                                      "cflavor": [cflavor.replace("\n", "\\n")], 
                                      "eflavor":[ecard["flavor_text"].replace("\n", "\\n")]})], ignore_index=True)
  
  if (not get_all) or (not response_json["has_more"]): break

  response = requests.get(response_json["next_page"])

  response_json = json.loads(response.text)


In [ ]:
df.head()

In [ ]:
df.to_csv(f'./out{word}{"-get_all" if get_all else ""}.csv', index=False)